In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from PIL import Image, ImageTk
import csv
from tkinter import  PhotoImage

model_var = None  # Déclaration de la variable model_var
df = None  # Déclaration de la variable df pour stocker les données chargées
fig_canvas = None  # Variable pour stocker la figure matplotlib

def preprocess_data(df):
    attack_group = {'Infilteration': 'Web attack', 
                    'Fuzzers': 'Web attack',
                    'Generic': 'Web attack',
                    'injection': 'Web attack', 
                    'Analysis': 'Web attack',
                    'xss': 'Web attack',
                    'Benign': 'BENIGN',
                    'backdoor': 'BACKDOOR', 
                    'scanning': 'SCANNING',
                    'mitm': 'MITM', 
                    'dos': 'DOS', 
                    'ddos': 'DDOS', 
                    'bruteforce': 'BRUTEFORCE', 
                    'Theft': 'THEFT', 
                    'Reconnaissance': 'RECON',
                    'Shellcode': 'SHELLCODE',
                    'ransomware': 'RANSOMWARE',
                    'Bot': 'BOT',
                    }
    df['Attack_Category'] = df['Attack'].map(lambda x: attack_group.get(x, 'UNKNOWN'))
    return df

def train_model(model_type, df):
    if model_type == "AdaBoost":
        model = AdaBoostClassifier(n_estimators=100, random_state=0) 
    elif model_type == "Decision Tree":
        model = DecisionTreeClassifier(random_state=0)
    elif model_type == "Gradient Boosting":
        model = GradientBoostingClassifier(n_estimators=100, random_state=30)
    elif model_type == "SVM":
        model = SVC(C=52)
    elif model_type == "RandomForest":
        model = RandomForestClassifier(n_estimators=11)
    elif model_type == "Logistic Regression":
        model = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=1000, random_state=0)
    elif model_type == "XGBoost":
        model = xgb.XGBClassifier(n_estimators=240)
    else:
        raise ValueError("Invalid model type. Choose from 'AdaBoost', 'Decision Tree', 'Gradient Boosting', 'SVM', 'RandomForest', 'Logistic Regression', or 'XGBoost'.")
        
    X = df.drop(['Label','Attack','Attack_Category'], axis=1).values 
    y = df.iloc[:, -3].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    data_train = pd.DataFrame(X_train)
    data_train["label"] = y_train
    
    epochs = 20
    val_score = []
    for i in range(epochs):
        shuffle(data_train)
        X_train = data_train.drop(["label"], axis=1).values
        y_train = data_train["label"]
        model.fit(X_train, y_train)
        score = model.score(X_train, y_train)
        val_score.append(score)

    plt.plot(val_score)
    plt.title('Validation', color='blue')
    plt.ylabel('accuracy', color='blue')
    plt.xlabel('nbr iteration d entrainement', color='blue')
    
    y_predict = model.predict(X_test)
    
    precision, recall, fscore, none = precision_recall_fscore_support(y_test, y_predict, average='weighted') 
    error_score = 1 - accuracy_score(y_test, y_predict)

    cm = confusion_matrix(y_test, y_predict)
    fig, ax = plt.subplots(figsize=(5, 5))
    sns.heatmap(cm, annot=True, linewidth=0.5, linecolor="red", fmt=".0f", ax=ax)
    ax.set_xlabel("y_pred")
    ax.set_ylabel("y_true")
    ax.set_xlabel('Prédit')
    ax.set_ylabel('Réel')
    ax.set_title(f'Matrice de Confusion \n{model_type}', color='blue')
    
    return val_score, error_score, precision, recall, fscore, fig

def load_csv():
    global df
    filepath = filedialog.askopenfilename(title="Choisir un fichier CSV", filetypes=[("Fichiers CSV", "*.csv")])
    if filepath:
        df = pd.read_csv(filepath)
        df = preprocess_data(df)            

def execute_model():
    
    global df, fig_canvas
    if df is not None:
        selected_model = model_var.get()
        try:
            result_label_img.pack_forget()
            clear_results()  # Effacer les résultats précédents s'il y en a

            val_score, error_score, precision, recall, fscore, fig = train_model(selected_model, df)
           
            # Affichage des résultats
            fig_canvas = FigureCanvasTkAgg(fig, master=result_frame)
            fig_canvas.draw()
            fig_canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
            result_label.config(text=f"Exactitude du modèle {selected_model}: {1 - error_score}\n"
                        f"Précision du modèle {selected_model}: {precision}\n"
                        f"Taux de détection du modèle {selected_model}: {recall}\n"  # Correction de la syntaxe
                        f"F1-score du modèle {selected_model}: {fscore}")            
            result_label.config(fg='blue')

            clear_button.pack(side=tk.BOTTOM, pady=5)  # Afficher le bouton Clear Results
        except ValueError as e:
            messagebox.showerror("Erreur", str(e))
    else:
        messagebox.showwarning("Avertissement", "Aucun fichier CSV chargé. Veuillez charger un fichier CSV avant d'exécuter le modèle.")
        

def clear_results():
    global fig_canvas
    if fig_canvas:
        fig_canvas.get_tk_widget().destroy()
        result_label_img.pack(pady=10, fill=tk.BOTH, expand=True)
        
    result_label.config(text="")  # Effacer le texte des résultats
    clear_button.pack_forget()  # Cacher le bouton Clear Results
    detection_result_frame.pack_forget()
    

def browse_file():
    filepath = filedialog.askopenfilename()
    if filepath:
        if df is None:
            messagebox.showerror("Erreur", "Aucun modèle n'a été exécuté. Veuillez exécuter au moins un modèle avant de procéder à la détection d'attaque.")
            return
        
        if result_label.cget("text") == "":
            messagebox.showerror("Erreur", "Aucun modèle n'a été exécuté. Veuillez exécuter au moins un modèle avant de procéder à la détection d'attaque.")

        else:
            with open(filepath, 'r') as file:
                reader = csv.DictReader(file)
                detection_result = ""
                for row in reader:
                    label_value = row.get('Label', 0)
                    if label_value == '1':
                        detection_result += "Attack\n"
                    else:
                        detection_result += "Non Attack\n"
                # Mise à jour du label dans le cadre de détection
                detection_label.config(text=detection_result)
                detect_attack_button.pack(pady=5)
                detection_result_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        
root = tk.Tk()
root.title("Model Selection and Results Visualization")
root.configure(bg='gray')  # Couleur de fond grise

# Obtenir la taille de l'écran
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
# Taille de la fenêtre (70% de la largeur et de la hauteur de l'écran)
width_percent = 100
height_percent = 100

window_width = (screen_width * width_percent) // 100
window_height = (screen_height * height_percent) // 100

# Définir la taille de la fenêtre
root.geometry(f"{window_width}x{window_height}")

# Création d'un cadre pour la barre de navigation
navbar_frame = tk.Frame(root, bg="black", bd=2, relief="raised")
navbar_frame.pack(side="top", fill="x")

# Ajout du titre "Detection" dans la barre de navigation
navbar_label = tk.Label(navbar_frame, text="Detection", bg="black", fg="white", font=("Arial", 14, "bold"))
navbar_label.pack(padx=10, pady=5)

# Centrer horizontalement le titre dans la barre de navigation
navbar_label.pack_configure(anchor="center")


# Logique pour l'onglet "Exécution des modèles"
model_var = tk.StringVar()
model_var.set("AdaBoost")  # Valeur initiale

# Frame pour la sélection du modèle
model_selection_frame = tk.Frame(root, bg='gray')
model_selection_frame.pack(side=tk.LEFT, fill=tk.Y, padx=2, pady=2)

# Widgets pour la sélection du modèle
tk.Label(model_selection_frame, text="Choisir un Modèle", bg='gray', padx=5, pady=5 ,font=("Arial", 12, "bold"), fg='orange').pack()
def change_text_color(event):
    event.widget.config(fg='orange')
    # Désélectionner les autres boutons radio et restaurer leur couleur par défaut
    for widget in model_selection_frame.winfo_children():
        if isinstance(widget, tk.Radiobutton) and widget != event.widget and widget.cget('fg') == 'orange':
            widget.config(fg='white')

def restore_default_color(event):
    if event.widget.cget('fg') != 'orange':
        event.widget.config(fg='white')

def change_model(event):
    selected_model = event.widget.cget('text')
    model_var.set(selected_model)
    change_text_color(event)

for model in ["AdaBoost", "Decision Tree", "Gradient Boosting", "SVM", "RandomForest", "Logistic Regression", "XGBoost"]:
    radio_button = tk.Radiobutton(model_selection_frame, text=model, variable=model_var, value=model, bg='gray', padx=5, pady=5, fg='white')
    radio_button.pack(anchor=tk.W)
    radio_button.bind("<Button-1>", change_model)
    radio_button.bind("<Leave>", restore_default_color)

# Frame pour afficher les résultats
result_frame = tk.Frame(root, bg='white', bd=2, relief='solid')  # Pas de taille fixe pour le cadre de résultats
result_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)



# Titre du cadre de résultats
result_title = tk.Label(result_frame, text="Résultat", bg='white', fg='black', font=("Arial", 12, "bold"  ))
result_title.pack(side=tk.TOP, pady=5)

result_label_img = tk.Label(result_frame, text="", bg='white')
result_label_img.pack(pady=10, fill=tk.BOTH, expand=True)

result_label = tk.Label(result_frame, text="", bg='white')
result_label.pack(pady=10, fill=tk.BOTH, expand=True)


# Charger l'image à afficher dans le label
image_path = r'C:\Users\hp\Music\Cyber-Security-PNG-Transparent-HD-Photo.png'
image = Image.open(image_path)
image = image.resize((400, 400))  # Redimensionner l'image selon vos besoins

# Convertir l'image en format compatible avec Tkinter
image_tk = ImageTk.PhotoImage(image)

# Configurer le label pour afficher l'image centrée
result_label_img.configure(image=image_tk, justify="center", compound="center")


# Ajout d'un cadre à côté du cadre de résultats pour afficher les résultats de la détection
detection_result_frame = tk.Frame(root, bg='white', bd=2, relief='solid')


# Titre du cadre de détection
detection_title = tk.Label(detection_result_frame, text="Résultats de Détection", bg='white', fg='black', font=("Arial", 12, "bold"))
detection_title.pack(side=tk.TOP, pady=5)

# Label pour afficher les résultats de détection
detection_label = tk.Label(detection_result_frame, text="", bg='white')
detection_label.pack(pady=20)

# Bouton Load Data
load_button = tk.Button(model_selection_frame, text="Charger les données", command=load_csv, bg='DarkSlateGray', fg='white', width=20)
load_button.pack(pady=5)

# Bouton Execute Model
execute_button = tk.Button(model_selection_frame, text="Exécuter", command=execute_model, bg='DarkSlateGray', fg='white', width=20)
execute_button.pack(pady=5)

# Bouton Clear Results (initialement caché)
clear_button = tk.Button(model_selection_frame, text="Effacer les résultats", command=clear_results, bg='DarkSlateGray', fg='white', width=20)
# Cacher le bouton Clear Results

# Bouton pour la détection d'attaque
detect_attack_button = tk.Button(model_selection_frame, text="Détection d'Attaque", command=browse_file, bg='DarkSlateGray', fg='white', width=20)
detect_attack_button.pack(pady=5)


# Charger l'image en utilisant PhotoImage
img = PhotoImage(file="D:\Downloads\cyber-security.png")

# Définir l'icône de la fenêtre
root.iconphoto(False, img)

root.mainloop()